In [13]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [14]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("training_data_all_offensive_offensive.csv")
print(data_initial.shape)

(95497, 951)


In [15]:
data_no_na = data_initial.dropna() # data should already have no NAs due to numerical imputation
data = data_no_na.select_dtypes(['number'])
x = data.loc[:, data.columns != 'net_xg']
x = x.loc[:, x.columns != 'net_xg']
y = data['net_xg']

X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.6, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, train_size = 0.5, test_size=0.5, random_state=45)

# Above two lines in unison accomplish a 60-20-20 split for train-validation-test

In [16]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 1000
val_batch_size = 1000
test_batch_size = 1000

cuda
cuda


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1:].values, dtype=torch.float32)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [18]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.batch_norm = nn.BatchNorm1d(self.hidden_size)

        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc4 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc5 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc6 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc7 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc8 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc9 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc10 = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        #print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc7(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc8(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc9(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc10(x)
        
        return x


In [19]:
import optuna

num_epochs = 20
trial_count = 30

learning_rate = 0.01
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.view(-1, 1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += train_batch_size
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        #model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for (X, y) in valid_loader:
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.view(-1, 1))
                val_loss += loss.float().item()
                count += val_batch_size
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 300, 600)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    print("learning rate:", lr)
    print("weight_decay:", weight_decay)
    print("hidden_size:", hidden_size)
    print("dropout_rate:", dropout_rate)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

study = optuna.create_study(direction="minimize")
input_dim = 950
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

[I 2023-05-04 23:51:15,981] A new study created in memory with name: no-name-c91492f6-bb3d-4c5a-a1b2-b8dd0337a2d4


learning rate: 0.03674814806929425
weight_decay: 0.0004526036659017309
hidden_size: 568
dropout_rate: 0.43430382020965186

EPOCH 1 of 20

Training loss: 0.001817930309936918
Validation loss: 0.0033675061345100404

EPOCH 2 of 20

Training loss: 0.0012536319740887346
Validation loss: 0.003128382205963135

EPOCH 3 of 20

Training loss: 0.001246760134039254
Validation loss: 0.003487733221054077

EPOCH 4 of 20

Training loss: 0.0012595355325731737
Validation loss: 0.003207119166851044

EPOCH 5 of 20

Training loss: 0.0012391197558107047
Validation loss: 0.003615252423286438

EPOCH 6 of 20

Training loss: 0.0012592647486719592
Validation loss: 0.003472531354427338

EPOCH 7 of 20

Training loss: 0.0012461512294308892
Validation loss: 0.003494594419002533

EPOCH 8 of 20

Training loss: 0.0012574317681378331
Validation loss: 0.0034937151551246645

EPOCH 9 of 20

Training loss: 0.0012286896541200834
Validation loss: 0.0035251589417457582

EPOCH 10 of 20

Training loss: 0.0012546118477295184
Vali

[I 2023-05-05 00:13:48,024] Trial 0 finished with value: 0.0036390867710113526 and parameters: {'lr': 0.03674814806929425, 'weight_decay': 0.0004526036659017309, 'hidden_size': 568, 'dropout_rate': 0.43430382020965186}. Best is trial 0 with value: 0.0036390867710113526.


Validation loss: 0.0036390867710113526
learning rate: 0.015345545327851273
weight_decay: 0.0002689256377212865
hidden_size: 388
dropout_rate: 0.42713668939327976

EPOCH 1 of 20

Training loss: 0.0002476311255374859
Validation loss: 1.810937556438148e-05

EPOCH 2 of 20

Training loss: 0.00011246209553089635
Validation loss: 1.7870855471119284e-05

EPOCH 3 of 20

Training loss: 0.00011360669521422222
Validation loss: 1.7039358522742985e-05

EPOCH 4 of 20

Training loss: 0.00011328611060463149
Validation loss: 1.826918739825487e-05

EPOCH 5 of 20

Training loss: 0.00011383215389375029
Validation loss: 1.686716233380139e-05

EPOCH 6 of 20

Training loss: 0.00011276284014356548
Validation loss: 2.0780552551150323e-05

EPOCH 7 of 20

Training loss: 0.00011275211490433791
Validation loss: 1.7247820366173982e-05

EPOCH 8 of 20

Training loss: 0.00010942411987945951
Validation loss: 1.6938427044078708e-05

EPOCH 9 of 20

Training loss: 0.00011362326312167891
Validation loss: 1.9344158563762902e

[I 2023-05-05 00:36:33,211] Trial 1 finished with value: 1.7381644574925304e-05 and parameters: {'lr': 0.015345545327851273, 'weight_decay': 0.0002689256377212865, 'hidden_size': 388, 'dropout_rate': 0.42713668939327976}. Best is trial 1 with value: 1.7381644574925304e-05.


...

Validation loss: 1.350326626561582e-05
learning rate: 0.009478715307618185
weight_decay: 4.804661683343479e-06
hidden_size: 456
dropout_rate: 0.4737741612908047

EPOCH 1 of 20

Training loss: 0.0001868148649047161
Validation loss: 5.157578140497208e-05

EPOCH 2 of 20

Training loss: 7.766003392893693e-05
Validation loss: 5.223467499017716e-05

EPOCH 3 of 20

Training loss: 7.801120689716833e-05
Validation loss: 5.1135057024657725e-05

EPOCH 4 of 20

Training loss: 7.788239689222698e-05
Validation loss: 5.1126769371330735e-05

EPOCH 5 of 20

Training loss: 7.796558583604878e-05
Validation loss: 5.32396586611867e-05

EPOCH 6 of 20

Training loss: 7.836370722487055e-05
Validation loss: 5.1943016052246094e-05

EPOCH 7 of 20

Training loss: 7.842723742641251e-05
Validation loss: 5.3074164129793646e-05

EPOCH 8 of 20

Training loss: 7.8344173857878e-05
Validation loss: 5.18940569832921e-05

EPOCH 9 of 20

Training loss: 7.89946033779917e-05
Validation loss: 5.140028465539217e-05

EPOCH 10 of

[I 2023-05-05 10:47:04,673] Trial 28 finished with value: 5.3554672189056874e-05 and parameters: {'lr': 0.009478715307618185, 'weight_decay': 4.804661683343479e-06, 'hidden_size': 456, 'dropout_rate': 0.4737741612908047}. Best is trial 23 with value: 1.3058469304814936e-05.


Validation loss: 5.3554672189056874e-05
learning rate: 0.0016640606882391846
weight_decay: 1.913234578271189e-06
hidden_size: 574
dropout_rate: 0.4394944666751021

EPOCH 1 of 20

Training loss: 0.0001309222342639134
Validation loss: 1.3906876742839814e-05

EPOCH 2 of 20

Training loss: 0.00010636854030448814
Validation loss: 1.3861664012074471e-05

EPOCH 3 of 20

Training loss: 0.00010457592290537111
Validation loss: 1.3963064085692167e-05

EPOCH 4 of 20

Training loss: 0.00010427907610247875
Validation loss: 1.4108075154945255e-05

EPOCH 5 of 20

Training loss: 0.00010441374187839443
Validation loss: 1.4257553778588772e-05

EPOCH 6 of 20

Training loss: 0.00010581142082810402
Validation loss: 1.4163893833756447e-05

EPOCH 7 of 20

Training loss: 0.00010636181654087428
Validation loss: 1.4027010649442674e-05

EPOCH 8 of 20

Training loss: 0.00010581581410149049
Validation loss: 1.3972263922914862e-05

EPOCH 9 of 20

Training loss: 0.00010435699247594537
Validation loss: 1.3903572130948

[I 2023-05-05 11:09:51,105] Trial 29 finished with value: 1.3987261429429054e-05 and parameters: {'lr': 0.0016640606882391846, 'weight_decay': 1.913234578271189e-06, 'hidden_size': 574, 'dropout_rate': 0.4394944666751021}. Best is trial 23 with value: 1.3058469304814936e-05.


Validation loss: 1.3987261429429054e-05
{'lr': 0.0016359428701992723, 'weight_decay': 3.4162832861696797e-06, 'hidden_size': 493, 'dropout_rate': 0.4524812840729844}


In [20]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

num_epochs = 100 # training for more epochs on best model (which will be saved)

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params2.pt")
# save the trained model
torch.save(model, "faceoffs_best_model2.pt")

{'lr': 0.0016359428701992723, 'weight_decay': 3.4162832861696797e-06, 'hidden_size': 493, 'dropout_rate': 0.4524812840729844}
Epoch 1 Training loss: 0.0673325229596732
Epoch 2 Training loss: 0.013592020785114888
Epoch 3 Training loss: 0.013600002428324058
Epoch 4 Training loss: 0.013778919755513298
Epoch 5 Training loss: 0.01375721810899418
Epoch 6 Training loss: 0.013892622953601953
Epoch 7 Training loss: 0.01392710790166567
Epoch 8 Training loss: 0.014003694458896744
Epoch 9 Training loss: 0.0139182370059706
Epoch 10 Training loss: 0.014036429435785475
Epoch 11 Training loss: 0.01347631866369268
Epoch 12 Training loss: 0.01316160180381146
Epoch 13 Training loss: 0.013135973396229333
Epoch 14 Training loss: 0.013145051703884685
Epoch 15 Training loss: 0.013145818391108307
Epoch 16 Training loss: 0.01312259836764685
Epoch 17 Training loss: 0.013122233016223743
Epoch 18 Training loss: 0.01312409233751482
Epoch 19 Training loss: 0.013154147770898095
Epoch 20 Training loss: 0.013176603852

In [21]:
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")


Test Loss: 0.0131
True labels: [0.1290534, 0.2789298, 0.0134995, 0.035773, 0.0841014, 0.0, 0.0139578, 0.2155861, 0.0, 0.0659834, 0.0364212, 0.0783474, 0.0866481, 0.011407, 0.0695776, 0.0290004, 0.0481963, 0.26799026, 0.0190991, 0.0125335, 0.0, 0.0, 0.5158027, 0.1583196, 0.0240285, 0.0, 0.0, 0.0, 0.681172, 0.0715155, 0.0, 0.0, 0.0548659, 0.0582331, 0.0, 0.018035, 0.0118242, 0.0217203, 0.22711492, 0.1427592, 0.0, 0.0126422, 0.0398317, 0.0, 0.10743066, 0.0444616, 0.05529905, 0.0459622, 0.0339057, 0.0374959, 0.0, 0.0, 0.1271679, 0.0, 0.00864218, 0.0504145, 0.0148984, 0.4786694, 0.0, 0.0, 0.0, 0.0, 0.0675065, 0.1154277, 0.0, 0.0, 0.0, 0.0122068, 0.0133577, 0.0, 0.0, 0.0, 0.06286565, 0.0255189, 0.0937638, 0.0, 0.0, 0.0181451, 0.0, 0.04734614, 0.0151993, 0.0168347, 0.0, 0.09682424, 0.0, 0.0, 0.06443419, 0.2162806, 0.0, 0.0, 0.0, 0.00501174, 0.0, 0.0, 0.0964286, 0.030284, 0.394822, 0.0155668, 0.0875407, 0.1123541, 0.18867794, 0.0, 0.046061, 0.0182286, 0.0722367, 0.0, 0.00814278, 0.0, 0.1229036

In [23]:
num_epochs = 20

learning_rate = 0.01
loss_func = nn.MSELoss()
input_dim = 799
best_params = {'lr': 0.008527084096275877, 'weight_decay': 6.071847158024609e-06, 'hidden_size': 352, 'dropout_rate': 0.4894615406787279}
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params_revisited.pt")
# save the trained model
torch.save(model, "faceoffs_best_model_revisited.pt")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x950 and 799x352)

In [ ]:
test_loss = 0
y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x799 and 529x352)